In [1]:
import time
import math
import random
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from sklearn.metrics import confusion_matrix
from datetime import timedelta

import dataset
import img_manipulation

%matplotlib inline

C:\ProgramFiles2\Anaconda3\envs\tensorflow_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Configuration

In [2]:
# paths
orginal_folder = "c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test"
processed_folder = "c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final_tr2/Test"
checkpoint_dir = "models/"

# Number of color channels for the images: 1 channel for gray-scale.
num_channels = 3

# image dimensions (only squares for now)
img_size = 224

# Size of image when flattened to a single dimension
img_size_flat = img_size * img_size * num_channels

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# class info
classes = dataset.get_classes(orginal_folder)
num_classes = len(classes)

# batch size
batch_size = 32

# validation split
validation_size = .16

# how long to wait after validation loss stops improving before terminating training
early_stopping = None  # use None if you don't want to implement early stoping

print("Classes: {num_classes}".format(num_classes = num_classes))
for key in classes:
    print("{class_name}".format(class_name=key))

Classes: 14
Achaemenid
Ancient
Art deco
Baroque
Byzantine
Chinese
Indo-islamic
Neoclassical
Postmodernism
Renesans
Romanesque
Russian Revival
Secession
Victorian + Quenn anne


# Image preprocesing
Resize and pad image to 224x224

In [3]:
img_manipulation.resize_and_pad_folder(orginal_folder, processed_folder, (224,224), 127)

Reading images
Processing Achaemenid
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\1.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\10.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\100.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\101.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\102.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\103.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\104.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\105.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\106.jpg
Exc

Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\77.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\78.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\79.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\8.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\80.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\81.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\82.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\83.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Achaemenid\84.jpg
Exception for file:c:/Users/Tomasz Marcinkowski

Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0062.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0063.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0064.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0065.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0066.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0067.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0068.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0069.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Ancient\0070.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documen

Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\103.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\104.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\105.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\106.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\107.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\108.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\109.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\11.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\110.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Document

Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\42.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\43.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\44.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\45.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\46.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\47.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\48.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\49.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zbiory/Zbior4_final/Test\Art deco\5.jpg
Exception for file:c:/Users/Tomasz Marcinkowski/Documents/MEGA/Zb

KeyboardInterrupt: 

# Resize Images to 224x224

In [ ]:
img_name = 'Baroque/0006.jpg'
orginal_image = cv2.imread(os.path.join(orginal_folder, img_name))
resized_image = cv2.imread(os.path.join(processed_folder, img_name))
plt.imshow(orginal_image)
plt.title('Before')
plt.show()
plt.imshow(resized_image)
plt.title('After')
plt.show()